# Collision checking (QP)

Do imports.

In [1]:
# Rigid body dynamics (pinocchio)
import pinocchio as pin

# Collision checking
import coal

# Visualization (meshcat)
from pinocchio.visualize import MeshcatVisualizer
import meshcat_shapes

# Robot models (robot_descriptions)
from robot_descriptions.loaders.pinocchio import load_robot_description

# Math ("expm" is the matrix exponential function)
import numpy as np
from scipy.linalg import expm, logm
from qpsolvers import solve_qp

# Timing
import time

# Plots
import matplotlib.pyplot as plt

# Interaction
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm

# Other utilities
import itertools
import copy

# Suppress the display of very small numbers
np.set_printoptions(suppress=True)

Load a robot model.

In [2]:
robot = load_robot_description(
    'iiwa14_description',           # name of robot model
    root_joint=None,                # fixed base
)

Remove all but one collision object for this example.

In [3]:
objs_to_remove = [obj.name for obj in robot.collision_model.geometryObjects if (obj.name != 'iiwa_link_5_0')]
for obj in objs_to_remove:
    robot.collision_model.removeGeometryObject(obj)

Add an obstacle.

In [4]:
# Define a function that specifies pose by x, y, z and yaw, pitch roll
def XYZYPRtoSE3(xyzypr):
    R = pin.utils.rotate('z', xyzypr[3]) @ pin.utils.rotate('y', xyzypr[4]) @ pin.utils.rotate('x', xyzypr[5])
    p = np.array(xyzypr[:3])
    return pin.SE3(R, p)

# Add obstacle to collision model
obs = pin.GeometryObject.CreateCapsule(0.1, 0.4)
obs.meshColor = robot.collision_model.geometryObjects[0].meshColor.copy()
obs.name = 'obs0'
obs.parentJoint = 0
obs.placement = XYZYPRtoSE3([-0.4, -0.0, 1.0, 0., 0., np.pi / 2])
idx = robot.collision_model.addGeometryObject(obs)
print(f'Added new geometry object with index {idx} to collision model.')

# Add obstacle to visual model
obs = copy.deepcopy(obs)
obs.meshColor = np.array([1.0, 0.2, 0.2, 1.0])
idx = robot.visual_model.addGeometryObject(obs)
print(f'Added new geometry object with index {idx} to visual model.')

Added new geometry object with index 1 to collision model.
Added new geometry object with index 15 to visual model.


Define function to convert each primitive geometry to a convex mesh by sampling its surface.

In [5]:
def primitive_to_convex(shape, n=32):
    """
    Convert a coal primitive shape to a ConvexBase by sampling
    its surface, building a BVHModel from the points, and computing
    the convex hull.
    """
    if isinstance(shape, coal.Box):
        # half-extents
        x, y, z = shape.halfSide
        verts = np.array([
            [ x,  y,  z], [ x,  y, -z], [ x, -y,  z], [ x, -y, -z],
            [-x,  y,  z], [-x,  y, -z], [-x, -y,  z], [-x, -y, -z],
        ])

    elif isinstance(shape, coal.Cylinder):
        r = shape.radius
        h = shape.halfLength
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        circle = np.column_stack([r * np.cos(theta), r * np.sin(theta)])
        top = np.column_stack([circle, np.full(n, h)])
        bot = np.column_stack([circle, np.full(n, -h)])
        verts = np.vstack([top, bot])

    elif isinstance(shape, coal.Cone):
        r = shape.radius
        h = shape.halfLength
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        base = np.column_stack([r * np.cos(theta), r * np.sin(theta),
                                np.full(n, -h)])
        apex = np.array([[0.0, 0.0, h]])
        verts = np.vstack([base, apex])

    elif isinstance(shape, coal.Sphere):
        verts = _sphere_points(shape.radius, n)

    elif isinstance(shape, coal.Ellipsoid):
        rx, ry, rz = shape.radii
        unit = _sphere_points(1.0, n)
        verts = unit * np.array([rx, ry, rz])

    elif isinstance(shape, coal.Capsule):
        r = shape.radius
        h = shape.halfLength
        # hemisphere + cylinder ring
        unit = _sphere_points(1.0, n)
        top_hemi = unit[unit[:, 2] >= 0] * r + [0, 0, h]
        bot_hemi = unit[unit[:, 2] <= 0] * r + [0, 0, -h]
        theta = np.linspace(0, 2 * np.pi, n, endpoint=False)
        ring = np.column_stack([r * np.cos(theta), r * np.sin(theta),
                                np.zeros(n)])
        verts = np.vstack([top_hemi, bot_hemi,
                           ring + [0, 0, h], ring + [0, 0, -h]])

    else:
        raise TypeError(f"Unsupported shape type: {type(shape)}")

    return _points_to_convex(verts)


def _sphere_points(radius, n):
    """Fibonacci sphere sampling."""
    indices = np.arange(n * n // 2)  # enough points
    # Use a simpler grid approach
    u = np.linspace(0, 2 * np.pi, n, endpoint=False)
    v = np.linspace(0, np.pi, n // 2 + 1)
    theta, phi = np.meshgrid(u, v)
    theta, phi = theta.ravel(), phi.ravel()
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)
    return np.column_stack([x, y, z])


def _points_to_convex(points):
    """Build a BVHModel from a point cloud and compute its convex hull."""
    bvh = coal.BVHModelOBBRSS()
    bvh.beginModel(0, len(points))
    bvh.addVertices(points)
    bvh.endModel()
    bvh.buildConvexHull(True, "Qt")
    return bvh.convex

Apply function to convert all collision geometries to convex meshes.

In [6]:
# Choose sampling density
n = 5

# Apply function
for geom_obj in robot.collision_model.geometryObjects:
    geom = geom_obj.geometry
    if isinstance(geom, coal.ConvexBase):
        print(f'No need to convert "{geom_obj.name}" (already a convex mesh)')
        continue  # already a convex mesh, nothing to do
    elif isinstance(geom, coal.BVHModelBase):
        print(f'Convert "{geom_obj.name}" (buildConvexHull)')
        geom.buildConvexHull(True, "Qt")
        geom_obj.geometry = geom.convex
    elif isinstance(geom, coal.ShapeBase):
        print(f'Convert "{geom_obj.name}" (primitive_to_convex)')
        geom_obj.geometry = primitive_to_convex(geom, n=n)
    else:
        print(f'Not sure what to do with "{geom_obj.name}"')

Convert "iiwa_link_5_0" (primitive_to_convex)
Convert "obs0" (primitive_to_convex)


Add all possible collision pairs, then list them. (There should be only one.)

In [7]:
# Add pairs
robot.collision_model.addAllCollisionPairs()

# List pairs
for i, pair in enumerate(robot.collision_model.collisionPairs):
    print(f'{i:2d} : {pair.first:2d} x {pair.second:2d} ({robot.collision_model.geometryObjects[pair.first].name} x {robot.collision_model.geometryObjects[pair.second].name})')

 0 :  0 x  1 (iiwa_link_5_0 x obs0)


Make all collision objects semi-transparent.

In [8]:
for go in robot.collision_model.geometryObjects:
    go.meshColor[-1] = 0.7

for go in robot.visual_model.geometryObjects:
    go.meshColor[-1] = 0.3

Update robot data.

In [9]:
robot.data = robot.model.createData()
robot.collision_data = robot.collision_model.createData()
robot.visual_data = robot.visual_model.createData()

Display robot (and obstacles) in browser.

In [10]:
# Create a visualizer
vis = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
robot.setVisualizer(vis, init=False)
vis.initViewer(open=True)
vis.loadViewerModel()

# Pause to allow meshcat initialization to finish
time.sleep(0.1)

# Choose what to display
vis.displayFrames(False)
vis.displayVisuals(True)
vis.displayCollisions(False)

# Add our own frames to the visualizer because the default frames are hard to see
frames_to_show = [
    'iiwa_link_0',
    'iiwa_link_1',
    'iiwa_link_2',
    'iiwa_link_3',
    'iiwa_link_4',
    'iiwa_link_5',
    'iiwa_link_6',
    'iiwa_link_7',
    'iiwa_link_ee_kuka',
]
for frame in frames_to_show:
    meshcat_shapes.frame(
        vis.viewer['frames/' + frame],
        opacity=0.5,
        axis_length=0.1,
        axis_thickness=0.0025,
        origin_radius=0.005,
    )

# Pause to allow meshcat initialization to finish
time.sleep(0.1)

# Change the camera view
vis.setCameraPosition(np.array([0., 2., 0.75]))
vis.setCameraTarget(np.array([0., 0., 0.75]))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7014/static/


Put the robot at its "neutral" configuration.

In [11]:
# Get and show the neutral configuration (most likely all zeros)
q = pin.neutral(robot.model)
print(f'{q = }')

# Do forward kinematics
# - Compute the placement of all joint frames (modifies robot.data but not robot.model or q)
pin.forwardKinematics(robot.model, robot.data, q)
# - Compute the placement of all link frames (modifies robot.data but not robot.model)
pin.updateFramePlacements(robot.model, robot.data)
# - Compute the placement of all collision geometry objects
pin.updateGeometryPlacements(
    robot.model,
    robot.data,
    robot.collision_model,
    robot.collision_data,
)

# Show the configuration in the visualizer
vis.display(q)
for frame in frames_to_show:
    frame_id = robot.model.getFrameId(frame)
    vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

q = array([0., 0., 0., 0., 0., 0., 0.])


Create sliders that allow us to change the configuration.

In [12]:
# Callback function that receives all slider values
def on_slider_change(change):
    global q
    q = np.array([slider.value for slider in sliders])
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    pin.updateGeometryPlacements(
        robot.model,
        robot.data,
        robot.collision_model,
        robot.collision_data,
    )
    vis.display(q)
    for frame in frames_to_show:
        frame_id = robot.model.getFrameId(frame)
        vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

# Create sliders for each joint
sliders = []
for i in range(robot.nq):
    slider = widgets.FloatSlider(
        value=0,
        min=robot.model.lowerPositionLimit[i],
        max=robot.model.upperPositionLimit[i],
        step=0.002,
        description=f'Joint {i+1}:',
        continuous_update=True,  # Only update on release
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='600px')  # Set slider width
    )
    # Attach the same callback to each slider
    slider.observe(on_slider_change, names='value')
    sliders.append(slider)

# Display all sliders
display(widgets.VBox(sliders))

# Initialize robot
on_slider_change(None)

Switch from showing "visual" geometry to showing "collision" geometry.

In [13]:
vis.displayVisuals(False)
vis.displayCollisions(True)
vis.viewer['frames'].set_property('visible', False)

Switch from smooth shading to flat shading for collision geometry.

In [14]:
# Import a sub-module of meshcat
import meshcat.geometry as mg

# Define a function that switches from smomoth shading to flat shading
def reload_with_flat_shading(vis, geom_model, geom_type, names=None):
    """Reload geometry objects with flat shading.
    
    If names is None, reloads all objects in geom_model.
    """
    _orig_init = mg.MeshPhongMaterial.__init__

    def _flat_init(self, **kwargs):
        kwargs.setdefault('flatShading', True)
        _orig_init(self, **kwargs)

    mg.MeshPhongMaterial.__init__ = _flat_init
    try:
        for go in geom_model.geometryObjects:
            if names is None or go.name in names:
                vis.loadViewerGeometryObject(go, geom_type)
    finally:
        mg.MeshPhongMaterial.__init__ = _orig_init

# Apply function to collision geometry
reload_with_flat_shading(vis, robot.collision_model, pin.GeometryType.COLLISION)

Show the mesh associated with each geometry object in the collision model as a wireframe. Also, show the body frame for each geometry object.

In [15]:
# Define a function that draws the wireframe associated with a mesh
def draw_mesh_wireframe(vis, robot, geom_name, geom_type=pin.GeometryType.COLLISION,
                        vertex_radius=0.0075, vertex_color=0x000000,
                        edge_color=0x000000, edge_opacity=0.5,
                        edge_width=0.002,
                        normal_length=0.05, normal_color=0x0000FF,
                        normal_width=0.0025,
                        show_frame=False,
                        frame_length=0.05, frame_thickness=0.0025, frame_origin_radius=0.0075):
    geom_model = (robot.collision_model if geom_type == pin.GeometryType.COLLISION
                  else robot.visual_model)
    idx = geom_model.getGeometryId(geom_name)
    go = geom_model.geometryObjects[idx]
    geom = go.geometry

    parent = vis.getViewerNodeName(go, geom_type)
    wire = vis.viewer[f'{parent}/wireframe']

    pts = np.array(geom.points(), copy=False)
    interior = pts.mean(axis=0)

    # Vertices
    for j, p in enumerate(pts):
        node = wire[f'v/{j}']
        meshcat_shapes.point(node, radius=vertex_radius, color=vertex_color)
        T = np.eye(4)
        T[:3, 3] = p
        node.set_transform(T)

    # Collect edges and face normals
    edges = set()
    centroids = []
    face_normals = []

    for i in range(geom.num_polygons):
        tri = geom.polygons(i)
        a, b, c = tri[0], tri[1], tri[2]
        edges.add((min(a, b), max(a, b)))
        edges.add((min(b, c), max(b, c)))
        edges.add((min(a, c), max(a, c)))

        pa, pb, pc = pts[a], pts[b], pts[c]
        centroid = (pa + pb + pc) / 3.0
        n = np.cross(pb - pa, pc - pa)
        norm = np.linalg.norm(n)
        if norm > 1e-12:
            n /= norm
            if n @ (centroid - interior) < 0:
                n = -n
            centroids.append(centroid)
            face_normals.append(n)

    def _cylinder_transform(p0, p1):
        d = p1 - p0
        length = np.linalg.norm(d)
        direction = d / length
        y = np.array([0., 1., 0.])
        T = np.eye(4)
        T[:3, 3] = (p0 + p1) / 2.0
        axis = np.cross(y, direction)
        sin_a = np.linalg.norm(axis)
        cos_a = y @ direction
        if sin_a > 1e-12:
            axis /= sin_a
            angle = np.arctan2(sin_a, cos_a)
            K = np.array([[0, -axis[2], axis[1]],
                          [axis[2], 0, -axis[0]],
                          [-axis[1], axis[0], 0]])
            T[:3, :3] = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * K @ K
        elif cos_a < 0:
            T[:3, :3] = -np.eye(3)
        return T, length

    # Edges
    edge_mat = mg.MeshPhongMaterial(color=edge_color, opacity=edge_opacity)
    if edge_opacity < 1.0:
        edge_mat.transparent = True
    for k, (i, j) in enumerate(edges):
        T, length = _cylinder_transform(pts[i], pts[j])
        wire[f'edges/{k}'].set_object(mg.Mesh(mg.Cylinder(length, edge_width), edge_mat))
        wire[f'edges/{k}'].set_transform(T)

    # Face normals
    normal_mat = mg.MeshPhongMaterial(color=normal_color)
    for k, (c, n) in enumerate(zip(centroids, face_normals)):
        T, length = _cylinder_transform(c, c + n * normal_length)
        wire[f'normals/{k}'].set_object(mg.Mesh(mg.Cylinder(length, normal_width), normal_mat))
        wire[f'normals/{k}'].set_transform(T)

    # Local coordinate frame
    if show_frame:
        meshcat_shapes.frame(wire['frame'], axis_length=frame_length,
                             axis_thickness=frame_thickness, origin_radius=frame_origin_radius)

# Apply the function to all geometry objects in the collision model
for go in robot.collision_model.geometryObjects:
    draw_mesh_wireframe(vis, robot, go.name, show_frame=True)

Pick a collision pair to use as an example (there is only one).

In [16]:
i_pair = 0

Define a function to get a description

$$ A q \leq b $$

of the halfspaces that define the convex mesh associated with a geometry object. This description should be consistent with points that are described in the coordinates of the *space* frame and not the body frame.

The convention used by pinocchio for each normal $n_i \in \R^3$ and each offset $o_i \in \R$ is to define the halfspace as

$$ \left\{ q \in \R^3 \colon n_i^T q + b_i \leq 0 \right\} $$

so we need to take

$$ A^\text{body} = \begin{bmatrix} n_1^T \\ \vdots \\ n_k^T \end{bmatrix}
\qquad\text{and}\qquad
b^\text{body} = \begin{bmatrix} -b_1 \\ \vdots \\ -b_k \end{bmatrix}. $$

In [17]:
def get_halfspaces(model, data, i):
    # Get normals and offsets in body frame
    normals = model.geometryObjects[i].geometry.normals()
    offsets = model.geometryObjects[i].geometry.offsets()

    # Get A and b in body frame
    A_b = normals
    b_b = -offsets

    # Get pose of body frame with respect to space frame
    R_w_b = data.oMg[i].rotation
    p_w_b = data.oMg[i].translation

    # Get A and b in space frame
    A_s = A_b @ R_w_b.T
    b_s = b_b + A_b @ R_w_b.T @ p_w_b
    return A_s, b_s

Define a function to compute the distance between two meshes in a collision pair by solving the quadratic program

$$
\begin{align*}
\underset{x_1, x_2 \in\R^3}{\text{minimize}} &\qquad \frac{1}{2}\| x_1 - x_2 \| \\
\text{subject to} &\qquad Gx \leq h \\
&\qquad A_1x_1 \leq b_1 \\
&\qquad A_2x_2 \leq b_2.
\end{align*}
$$

Remember that `solve_qp` expects the QP to be expressed in the following standard form:

$$
\begin{align*}
\underset{x\in\R^n}{\text{minimize}} &\qquad \frac{1}{2}x^TPx + q^Tx \\
\text{subject to} &\qquad Gx \leq h \\
&\qquad Ax=b \\
&\qquad x_\text{min} \leq x \leq x_\text{max}.
\end{align*}
$$

In [18]:
def compute_distance(model, data, i_pair, eps_abs=1e-8):
    # Get halfspaces
    A1, b1 = get_halfspaces(model, data, model.collisionPairs[i_pair].first)
    A2, b2 = get_halfspaces(model, data, model.collisionPairs[i_pair].second)
    
    # Set up objective and constraints
    M = np.block([[np.identity(3), -np.identity(3)]])
    G = np.block([[A1, np.zeros((A1.shape[0], A2.shape[1]))],
                [np.zeros((A2.shape[0], A1.shape[1])), A2]])
    h = np.concatenate([b1, b2])

    # Solve
    x = solve_qp(M.T @ M, np.zeros(6), G, h, solver='proxqp', verbose=False, eps_abs=eps_abs)

    # Parse solution
    x1 = x[:3]
    x2 = x[3:]

    # Get minimum distance
    d = np.linalg.norm(x2 - x1)
    
    return d, x1, x2

Apply the function to compute the distance to collision (as well as the two "witness points" $x_1$ and $x_2$).

In [19]:
d_qp, x1_qp, x2_qp = compute_distance(robot.collision_model, robot.collision_data, i_pair)

Compute the same distance (and the same witness points) using pinocchio.

In [20]:
result = pin.computeDistance(robot.collision_model, robot.collision_data, i_pair)
d_pin = result.min_distance
x1_pin = result.getNearestPoint1()
x2_pin = result.getNearestPoint2()

Compare results.

In [21]:
print(f'  by hand: {d_qp:10.8f}')
print(f'pinocchio: {d_pin:10.8f}')

assert(np.isclose(d_qp, d_pin, atol=1e-5, rtol=1e-3))

  by hand: 0.23009694
pinocchio: 0.23000000


Define a function to draw the closest points (the "witness points") that correspond to a collision pair.

In [22]:
def draw_closest_points(vis, x1, x2, name='closest_points',
                        point_radius=0.01, point_color=0xFF0000,
                        line_width=0.002, line_color=0xFF0000):
    node = vis.viewer[name]

    # Point x1
    meshcat_shapes.point(node['x1'], radius=point_radius, color=point_color)
    T1 = np.eye(4)
    T1[:3, 3] = x1
    node['x1'].set_transform(T1)

    # Point x2
    meshcat_shapes.point(node['x2'], radius=point_radius, color=point_color)
    T2 = np.eye(4)
    T2[:3, 3] = x2
    node['x2'].set_transform(T2)

    # Line segment between them
    d = x2 - x1
    length = np.linalg.norm(d)
    if length > 1e-12:
        direction = d / length
        y = np.array([0., 1., 0.])
        T = np.eye(4)
        T[:3, 3] = (x1 + x2) / 2.0
        axis = np.cross(y, direction)
        sin_a = np.linalg.norm(axis)
        cos_a = y @ direction
        if sin_a > 1e-12:
            axis /= sin_a
            angle = np.arctan2(sin_a, cos_a)
            K = np.array([[0, -axis[2], axis[1]],
                          [axis[2], 0, -axis[0]],
                          [-axis[1], axis[0], 0]])
            T[:3, :3] = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * K @ K
        elif cos_a < 0:
            T[:3, :3] = -np.eye(3)
        mat = mg.MeshPhongMaterial(color=line_color)
        node['line'].set_object(mg.Mesh(mg.Cylinder(length, line_width), mat))
        node['line'].set_transform(T)

Create new sliders with new callbacks that visualize the results of distance computation.

In [23]:
color_illini_orange = 0xE84A27
color_illini_blue = 0x13294B
color_harvest = 0xFCB316

# Text display widget
distance_label = widgets.HTML(
    value='',
    layout=widgets.Layout(width='600px')
)

# Callback function that receives all slider values
def on_slider_change(change):
    # Get configuration from sliders
    global q
    q = np.array([slider.value for slider in sliders])

    # Do forward kinematics
    pin.forwardKinematics(robot.model, robot.data, q)
    pin.updateFramePlacements(robot.model, robot.data)
    pin.updateGeometryPlacements(robot.model, robot.data,
                                 robot.collision_model, robot.collision_data)
    
    # Update display
    vis.display(q)
    for frame in frames_to_show:
        frame_id = robot.model.getFrameId(frame)
        vis.viewer['frames/' + frame].set_transform(robot.data.oMf[frame_id].homogeneous)

    # QP distance computation (mine)
    d_qp, x1_qp, x2_qp = compute_distance(robot.collision_model, robot.collision_data, i_pair)
    draw_closest_points(
        vis, x1_qp, x2_qp, name='closest_qp',
        point_color=color_illini_orange, line_color=color_illini_orange,
        point_radius=0.02, line_width=0.005,
    )

    # Coal distance computation (pinocchio)
    result = pin.computeDistance(robot.collision_model, robot.collision_data, i_pair)
    draw_closest_points(
        vis, result.getNearestPoint1(), result.getNearestPoint2(), name='closest_pin',
        point_color=color_harvest, line_color=color_harvest,
        point_radius=0.019, line_width=0.0049,
    )

    # Update text display
    distance_label.value = (
        f'<pre style="font-size:18px">'
        f'<span style="color:#{color_illini_orange:06X}">■</span>   QP distance: {d_qp:13.8f}\n'
        f'<span style="color:#{color_harvest:06X}">■</span> Coal distance: {result.min_distance:13.8f}'
        f'</pre>'
    )

# Create sliders for each joint
sliders = []
for i in range(robot.nq):
    slider = widgets.FloatSlider(
        value=0,
        min=robot.model.lowerPositionLimit[i],
        max=robot.model.upperPositionLimit[i],
        step=0.002,
        description=f'Joint {i+1}:',
        continuous_update=True,  # Only update on release
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
        layout=widgets.Layout(width='600px')  # Set slider width
    )
    # Attach the same callback to each slider
    slider.observe(on_slider_change, names='value')
    sliders.append(slider)

# Display sliders and distance label
display(widgets.VBox(sliders + [distance_label]))

# Initialize robot
on_slider_change(None)